In [26]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

cats = ['x01','x02','x03','x04','x05','x06','x07','x08','x09','x10']
catsname = ['城市人口密度','城镇人口比例','城市建成区面积占总面积比例','人均城市道路面积','不透水面面积占总面积比例','人均GDP','第二三产业增加值占GDP比重','人均社会消费品零售总额','万人拥有医疗机构床位','万人拥有医生人数']
pcs = ['pc01','pc02','pc03','pc04','pc05','pc06','pc07','pc08','pc09','pc10']

results = pd.DataFrame()
percents = pd.DataFrame()

df = pd.read_excel('D:\ee-composite\src\sqlserver\Data_U_1.xlsx')
# print(df,df.shape)
df1 = df.query('catid=="x01"').sort_values('sn')
for yyyy in range(1985,2021):
    x = df1.loc[:,['sn','cityen','province','city']]
    x['year'] = yyyy
    results = results.append(x)
results = results.reset_index()   
results.drop('index', axis=1, inplace=True) 
# print(results,results.shape)

for i in range(1,11,1):
    cc = 'x0'+str(i) if i<10 else 'x'+str(i)
    df1 = df.query('catid=="'+cc+'"').sort_values('sn')
    tmp = pd.DataFrame()
    for yyyy in range(1985,2021):
        tmp = tmp.append(df1.loc[:,[yyyy]].rename(columns={yyyy:cc}))
    tmp = tmp.reset_index()[cc]
    tmp = (tmp-tmp.mean())/tmp.std()
    tmp = (tmp-tmp.min())/(tmp.max()-tmp.min())
    results = pd.concat([results,tmp],axis=1)
# print(results,results.shape)

for i in x['sn']:
    pcadata = results.query('sn=='+str(i)).sort_values('year').iloc[:,5:]
    # print(pcadata,pcadata.shape)

    pca = PCA(n_components=10,whiten=False)
    tmp = pca.fit_transform(pcadata)
    pcp =  pd.DataFrame(pca.explained_variance_ratio_).T
    pcp.columns = pcs
    # print(pcp,pcp.shape)
    pcp['sn'] = pd.Series([i])
    pcp['city'] = pd.Series([x.iat[i-1,1]])
    pcp['citycn'] = pd.Series(x.iat[i-1,3])
    pcp['province'] = pd.Series(x.iat[i-1,2])
    percents = pd.concat([percents,pcp])

print(percents,percents.shape)
print(percents.mean(),percents.std())
print(percents.groupby(['sn','city','citycn']).mean())

percents.groupby(['sn','city','citycn']).mean().to_excel('Data_U_1_explain.xlsx', sheet_name='explain')

       pc01      pc02      pc03      pc04      pc05      pc06      pc07  \
0  0.914219  0.069373  0.011711  0.002542  0.001450  0.000481  0.000131   
0  0.899671  0.068321  0.015438  0.010209  0.002459  0.001934  0.001022   
0  0.928855  0.060388  0.005700  0.003800  0.000494  0.000357  0.000195   
0  0.905665  0.080572  0.007555  0.003755  0.001743  0.000596  0.000097   
0  0.873678  0.089570  0.023442  0.006096  0.005447  0.001130  0.000406   
0  0.960731  0.023790  0.008491  0.004149  0.001471  0.000842  0.000241   
0  0.910728  0.070604  0.008751  0.006138  0.002900  0.000340  0.000244   
0  0.871239  0.074538  0.022470  0.013863  0.010840  0.002705  0.002324   
0  0.798324  0.149307  0.036114  0.009741  0.003415  0.001547  0.000916   
0  0.868864  0.122197  0.003803  0.003071  0.001328  0.000676  0.000039   
0  0.938300  0.038184  0.021241  0.001730  0.000210  0.000147  0.000112   
0  0.881194  0.080038  0.027216  0.007674  0.002201  0.001197  0.000232   
0  0.937632  0.041275  0.